<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [3]:
from __future__ import print_function

import tensorflow
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [4]:
max_features = 20000
maxlen = 80
batch_size = 32

In [5]:
import newspaper

billy = newspaper.build('http://www.gutenberg.org/files/100/100-0.txt')

E0819 15:57:59.464298 123145557037056 network.py:112] CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: http://www.gutenberg.org/feed
E0819 15:57:59.464808 123145562292224 network.py:112] CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: http://www.gutenberg.org/rss


In [10]:
print(billy)

In [28]:
import pandas as pd

billy = pd.read_fwf('http://www.gutenberg.org/files/100/100-0.txt')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 153, saw 3


In [ ]:
billy.shape

In [14]:
billy.head(20)

,Unnamed: 0
"Project Gutenberg’s The Complete Works of William Shakespeare, by William",NaN
Shakespeare,NaN
NaN,NaN
This eBook is for the use of anyone anywhere in the United States and,NaN
most other parts of the world at no cost and with almost no restrictions,NaN
"whatsoever. You may copy it, give it away or re-use it under the terms",NaN
of the Project Gutenberg License included with this eBook or online at,NaN
"www.gutenberg.org. If you are not located in the United States, you’ll",NaN
have to check the laws of the country where you are located before using,NaN
this ebook.,NaN


In [15]:
billy = billy[23:]

In [18]:
billy = billy[20:]

In [22]:
billy = billy.drop(columns=['Unnamed: 0'])

In [24]:
billy = billy.dropna()

In [25]:
billy.shape

(164158, 0)

In [27]:
billy.head(20)

""
THE SONNETS
NaN
ALL’S WELL THAT ENDS WELL
NaN
THE TRAGEDY OF ANTONY AND CLEOPATRA
NaN
AS YOU LIKE IT
NaN
THE COMEDY OF ERRORS
NaN


In [36]:
billy = newspaper.build('http://www.gutenberg.org/files/100/100-0.txt')

E0819 16:13:41.239121 123145556500480 network.py:112] CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: http://www.gutenberg.org/feed
E0819 16:13:41.239580 123145567010816 network.py:112] CRITICAL - [REQUEST FAILED] 404 Client Error: Not Found for url: http://www.gutenberg.org/rss


In [38]:
billy

In [46]:
billy_text = ''

for article in billy.articles:
    try:
        article.download()
        article.parse()
        billy_text += '\n\n' + article.text
    except:
        print('failed:' + article.url)
        
billy_text = billy_text.split('\n\n')
print(billy_text)

AttributeError: '_io.TextIOWrapper' object has no attribute 'articles'

In [41]:
billy = open('/Users/mattmastin/Desktop/100-0.txt')

In [44]:
# billy.read()

In [47]:
import numpy as np

iteration = 1000
sequence_length = 40
batch_size = 100
hidden_size = 500
learning_rate = 1e-1

W_xh = np.random.randn(hidden_size, 100) * 0.01
W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
W_hy = np.random.randn(100, hidden_size) * 0.01

b_h = np.zeros((hidden_size, 1))
b_y = np.zeros((100, 1))

h_prev = np.zeros((hidden_size, 1))

In [48]:
def forwardprop(inputs, targets, h_prev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(h_prev)
    loss = 0
    
    for t in range(len(inputs)):
        xs[t] = np.zeros((100, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h)
        ys[t] = np.dot(W_hy, hs[t]) + b_y
        ps[t] = np.exp(ys[t] / np.sum(np.exp(ys[t])))
        
        loss += -np.log(ps[t][targets[t], 0])
        
    return loss, ps, hs, xs

In [49]:
def backprop(ps, inputs, hs, xs, targets):
    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = npcopy(ps[t])
        dy[targets[t]]
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(W_hy.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
        
    return dWxh, dWhh, dWhy, dbh, dby
        

In [50]:
data_pointer = 0

mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y)

for i in range(iteration):
    h_prev = np.zeros((hidden_size, 1))
    data_pointer = 0
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in article_text[data_pointer:data_pointer + sequence_length]]
        targets = [char_to_int[ch] for ch in article_text[data_pointer + 1: data_pointer + sequence_length + 1]]
        
        if (data_pointer + sequence_length + 1 >= len(article_text) and b == batch_size-1):
            targets.append(char_to_int[" "])
            
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
        
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs, targets)
        
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
            
        data_pointer += sequence_length
        
    if i % 100 == 0:
        print('iter %d, loss: %f' % (i, loss))

NameError: name 'article_text' is not defined

In [51]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1))
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size, 1))
    
    for ti in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h)
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(num_chars), p = p.ravel())
        x = np.zeros((num_chars, 1))
        x[ix] = 1
        ixes.append(ix)
    txt = test_char + ''.join(int_to_char[i] for i in ixes)
    print('----\n %s \n----' % (txt, ))

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN